# Versione originale con UCT


In [4]:
import random
from math import sqrt, log
import uuid
import gym
from gym.envs.registration import register
import json

class Node:

    def __init__(self, state, action, action_space, reward, terminal):
        self.identifier = str(uuid.uuid1())
        self.parent_identifier = None
        self.children_identifiers = []
        self.untried_actions = list(range(action_space))
        self.state = state
        self.total_simulation_reward = 0
        self.num_visits = 0
        self.performance = 0
        self.action = action
        self.reward = reward
        self.terminal = terminal

    def __str__(self):
        return "{}: (action={}, visits={}, reward={:d}, ratio={:0.4f})".format(
                                                  self.state,
                                                  self.action,
                                                  self.num_visits,
                                                  int(self.total_simulation_reward),
                                                  self.performance)

    def untried_action(self):
        action = random.choice(self.untried_actions)
        self.untried_actions.remove(action)
        return action


def vertical_lines(last_node_flags):
    vertical_lines = []
    vertical_line = '\u2502'
    for last_node_flag in last_node_flags[0:-1]:
        if last_node_flag == False:
            vertical_lines.append(vertical_line + ' ' * 3)
        else:
            # space between vertical lines
            vertical_lines.append(' ' * 4)
    return ''.join(vertical_lines)

def horizontal_line(last_node_flags):
    horizontal_line = '\u251c\u2500\u2500 '
    horizontal_line_end = '\u2514\u2500\u2500 '
    if last_node_flags[-1]:
        return horizontal_line_end
    else:
        return horizontal_line

class Tree:

    def __init__(self):
        self.nodes = {}
        self.root = None

    def size(self):
        """Returns the number of nodes in the tree."""
        return len(self.nodes)

    def depth_info(self):
        """Calculates the maximum depth and average depth of the tree."""
        depths = []

        def calculate_depth(node, current_depth):
            depths.append(current_depth)
            for child_id in node.children_identifiers:
                calculate_depth(self.nodes[child_id], current_depth + 1)

        if self.root is not None:
            calculate_depth(self.root, 0)

        if depths:
            max_depth = max(depths)
            avg_depth = sum(depths) / len(depths)
            return max_depth, avg_depth
        else:
            return 0, 0  # In case the tree is empty

    def is_expandable(self, node):
        if node.terminal:
            return False
        if len(node.untried_actions) > 0:
            return True
        return False

    def iter(self, identifier, depth, last_node_flags):
        if identifier is None:
            node = self.root
        else:
            node = self.nodes[identifier]

        if depth == 0:
            yield "", node
        else:
            yield vertical_lines(last_node_flags) + horizontal_line(last_node_flags), node

        children = [self.nodes[identifier] for identifier in node.children_identifiers]
        last_index = len(children) - 1

        depth += 1
        for index, child in enumerate(children):
            last_node_flags.append(index == last_index)
            for edge, node in self.iter(child.identifier, depth, last_node_flags):
                yield edge, node
            last_node_flags.pop()

    def add_node(self, node, parent=None):
        self.nodes.update({node.identifier: node})

        if parent is None:
            self.root = node
            self.nodes[node.identifier].parent = None
        else:
            self.nodes[parent.identifier].children_identifiers.append(node.identifier)
            self.nodes[node.identifier].parent_identifier=parent.identifier

    def children(self, node):
        children = []
        for identifier in self.nodes[node.identifier].children_identifiers:
            children.append(self.nodes[identifier])
        return children

    def parent(self, node):
        parent_identifier = self.nodes[node.identifier].parent_identifier
        if parent_identifier is None:
            return None
        else:
            return self.nodes[parent_identifier]

    def show(self):
        lines = ""
        for edge, node in self.iter(identifier=None, depth=0, last_node_flags=[]):
            lines += "{}{}\n".format(edge, node)
        print(lines)


class MonteCarloTreeSearchUCT():

    def __init__(self, env, tree, exploration_constant=1.4):
        self.env = env
        self.tree = tree
        self.action_space = self.env.action_space.n
        self.exploration_constant = exploration_constant
        state = self.env.reset()
        self.tree.add_node(Node(state=state, action=None, action_space=self.action_space, reward=0, terminal=False))
        # Initialize reward statistics
        self.rewards = []

    def expand(self, node):
        action = node.untried_action()
        state, reward, done, _, _ = self.env.step(action)
        new_node = Node(state=state, action=action, action_space=self.action_space, reward=reward, terminal=done)
        self.tree.add_node(new_node, node)
        return new_node

    def default_policy(self, node):
        if node.terminal:
            return node.reward

        total_reward = 0
        num_steps = 0
        while True:
            action = random.randint(0, self.action_space-1)
            state, reward, done, _, _ = self.env.step(action)
            total_reward += reward
            num_steps += 1
            if done:
                break
        average_reward = total_reward / num_steps if num_steps > 0 else 0
        self.rewards.append(average_reward)
        return average_reward

    def compute_value(self, parent, child):
        exploitation_term = child.total_simulation_reward / child.num_visits
        exploration_term = self.exploration_constant * sqrt(2 * log(parent.num_visits) / child.num_visits)
        return exploitation_term + exploration_term

    def best_child(self, node):
        best_child = self.tree.children(node)[0]
        best_value = self.compute_value(node, best_child)
        iter_children = iter(self.tree.children(node))
        next(iter_children)
        for child in iter_children:
            value = self.compute_value(node, child)
            if value > best_value:
                best_child = child
                best_value = value
        return best_child

    def tree_policy(self):
        node = self.tree.root
        while not node.terminal:
            if self.tree.is_expandable(node):
                return self.expand(node)
            else:
                node = self.best_child(node)
                state, reward, done, _, _ = self.env.step(node.action)
                assert node.state == state
        return node

    def backward(self, node, value):
        while node:
            node.num_visits += 1
            node.total_simulation_reward += value
            node.performance = node.total_simulation_reward/node.num_visits
            node = self.tree.parent(node)

    def forward(self):
        self._forward(self.tree.root)

    def _forward(self,node):
        best_child = self.best_child(node)

        print("****** {} ******".format(best_child.state))

        for child in self.tree.children(best_child):
            print("{}: {:0.4f}".format(child.state, child.performance))

        if len(self.tree.children(best_child)) > 0:
            self._forward(best_child)

    def render_policy(self, map_size):
        """
        Render the final policy obtained from the MCTS process using the gym's render function.
        """
        node = self.tree.root
        path = []
        directions = {0: 'Left', 1: 'Down', 2: 'Right', 3: 'Up'}
        
        print("Rendering final policy...\n")
        
        # Close the current environment if open and reset with rendering enabled
        self.env.close()
        self.env = gym.make('FrozenLake-v1', is_slippery=False, render_mode='human', map_name=map_size)
        self.env.reset()

        # Traverse the tree to follow the best path
        while node and not node.terminal:
            self.env.render()
            path.append(node.state)
            best_child = max(self.tree.children(node), key=lambda n: n.performance, default=None)
            if best_child is None:
                break
            if best_child.action is not None:
                print(f"Action: {directions[best_child.action]} -> State: {best_child.state}")
                self.env.step(best_child.action)  # Take the action in the environment
            node = best_child

        # Final rendering after reaching the goal
        self.env.render()
        self.env.close()    
        print(f"\nFinal policy path (states): {path}")
        
        path2 = [item[0] if isinstance(item, tuple) else item for item in path]

        print(path2)
        return path2



In [5]:
import gym
from gym.envs.registration import register

def init_env():
    register(
        id='FrozenLakeNotSlippery-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name': '4x4', 'is_slippery': False}
    )
    return gym.make('FrozenLakeNotSlippery-v0')

def main():
    random.seed(2)
    env = init_env()
    steps_values = [1000, 5000, 10000]  # Example steps values
    exploration_constants = [0.5, 1.0/sqrt(2.0), 1.4, 2.0]  # Example exploration constants

    results = []

    for steps in steps_values:
        for exploration_constant in exploration_constants:
            print(f"\nRunning MCTS with exploration constant: {exploration_constant} and steps: {steps}\n")

            tree = Tree()
            monteCarloTreeSearch = MonteCarloTreeSearchUCT(env=env, tree=tree, exploration_constant=exploration_constant)

            for _ in range(0, steps):
                env.reset()
                node = monteCarloTreeSearch.tree_policy()
                reward = monteCarloTreeSearch.default_policy(node)
                monteCarloTreeSearch.backward(node, reward)

            # Measure and display tree statistics
            tree_size = tree.size()
            max_depth, avg_depth = tree.depth_info()
            max_reward = max(monteCarloTreeSearch.rewards, default=0)
            avg_reward = sum(monteCarloTreeSearch.rewards) / len(monteCarloTreeSearch.rewards) if monteCarloTreeSearch.rewards else 0

            print(f"Tree size: {tree_size}")
            print(f"Maximum depth: {max_depth}")
            print(f"Average depth: {avg_depth:.2f}")
            print(f"Maximum reward: {max_reward}")
            print(f"Average reward: {avg_reward:.2f}")

            monteCarloTreeSearch.tree.show()
            monteCarloTreeSearch.forward()
            policy_path = monteCarloTreeSearch.render_policy("4x4")

            print(f"Final policy path with exploration constant {exploration_constant}: {policy_path}")

            # Save the results for this combination
            results.append({
                'exploration_constant': exploration_constant,
                'steps': steps,
                'tree_size': tree_size,
                'max_depth': max_depth,
                'avg_depth': avg_depth,
                'max_reward': max_reward,
                'avg_reward': avg_reward,
                'policy_path': policy_path
            })

    # Save results to a JSON file
    with open('simulations/mcts_uct_results.json', 'w') as json_file:
        json.dump(results, json_file, indent=4)

if __name__ == "__main__":
    main()


c:\Users\cater\anaconda3\envs\cidl\Lib\site-packages\gym\envs\registration.py:498: UserWarning: WARN: Overriding environment FrozenLakeNotSlippery-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")



Running MCTS with exploration constant: 0.5 and steps: 1000

Tree size: 710
Maximum depth: 5
Average depth: 4.47
Maximum reward: 0.25
Average reward: 0.00
(0, {'prob': 1}): (action=None, visits=1000, reward=1, ratio=0.0014)
├── 0: (action=0, visits=245, reward=0, ratio=0.0003)
│   ├── 1: (action=2, visits=61, reward=0, ratio=0.0013)
│   │   ├── 2: (action=2, visits=15, reward=0, ratio=0.0051)
│   │   │   ├── 1: (action=0, visits=4, reward=0, ratio=0.0000)
│   │   │   │   ├── 1: (action=3, visits=1, reward=0, ratio=0.0000)
│   │   │   │   ├── 2: (action=2, visits=1, reward=0, ratio=0.0000)
│   │   │   │   └── 0: (action=0, visits=1, reward=0, ratio=0.0000)
│   │   │   ├── 2: (action=3, visits=4, reward=0, ratio=0.0192)
│   │   │   │   ├── 6: (action=1, visits=1, reward=0, ratio=0.0000)
│   │   │   │   ├── 3: (action=2, visits=1, reward=0, ratio=0.0000)
│   │   │   │   └── 1: (action=0, visits=1, reward=0, ratio=0.0000)
│   │   │   ├── 6: (action=1, visits=3, reward=0, ratio=0.0000)
│  